In [1]:
import pandas as pd #importing libraries
import os
import numpy as np
from datetime import datetime, timedelta

In [51]:
path_noise = ("noise_path") #defind path to read noise files
path_EDA = ("EDA_path") #define path to read EDA files
target = ("stat_path") #Statistics target
clean_tar = ("clean_path") #define path to noise-clean EDA's

In [5]:
noise_percentage = pd.DataFrame({'EDA_num':[],'binary[%]':[],'multiple[%]':[]}) #create a file to calculate noise statistics

In [23]:
for item_E in os.listdir(path_EDA): #for every item in the EDA diractory:
    index = item_E.find("_") 
    sub_number = item_E[:index] #find its subject number by index of the first "_"
    #then, search for the matching Binary noise file by the subject number
    for item_N in os.listdir(path_noise):
        if item_N.startswith(sub_number+"_") and item_N.endswith("Binary.csv"):

            noise= pd.read_csv(path_noise+"\\"+item_N) #read the right noise
            EDA = pd.read_csv(path_EDA+"\\"+item_E) #read the right EDA file
            
            eda_len = len(EDA) - 2 #calculate EDA duration by rows (-2 because the real measurment starts at row 3)
            
            noisy_parts = noise[noise["BinaryLabels"] ==-1][["StartTime", "EndTime"]] #search for the noisy parts
            noise_len= len(noisy_parts) * 20 #calculate duration of noise (*20 because every noise epoc is 5 sec, so it is 20 rows of 0.250 millisec)
            
            percent = float((noise_len/eda_len)*100) #calculate Binary percent
            
            new_row = {'EDA_num':sub_number,'binary[%]':percent}
            noise_percentage = noise_percentage.append(new_row, ignore_index=True) #Add the new rows to the statistics dataframe

In [24]:
for item_E in os.listdir(path_EDA): #for every item in the EDA diractory:
    index = item_E.find("_") 
    sub_number = item_E[:index] #find its subject number by index of the first "_"
         #then, search for the matching Multiclass noise file by the subject number
    for item_N in os.listdir(path_noise):
        if item_N.startswith(sub_number+"_") and item_N.endswith("Multiclass.csv"):

            noise= pd.read_csv(path_noise+"\\"+item_N) #read the right noise
            EDA = pd.read_csv(path_EDA+"\\"+item_E) #read the right EDA file
            
            eda_len = len(EDA) - 2 #calculate EDA duration by rows (-2 because the real measurment starts at row 3)
            
            noisy_parts = noise[noise["MulticlassLabels"] ==-1][["StartTime", "EndTime"]] #search for the noisy parts (only sure)
            noise_len= len(noisy_parts) * 20 #calculate duration of noise (*20 because every noise epoc is 5 sec, so it is 20 rows of 0.250 millisec)
            
            percent = float((noise_len/eda_len)*100) #calculate multiclass percent
            
            noise_percentage.loc[noise_percentage.EDA_num == sub_number, "multiple[%]"] = percent #update the multiclass column by the EDAnumber

In [26]:
noise_percentage.to_csv(target) #save the statistics table

In [40]:
def cleaning_EDA_bin(path_EDA, path_noise, clean_tar): #define cleaning function
    
    tar = pd.read_csv(target) # Read the statistics file
    for item_E in os.listdir(path_EDA): #for every item in the EDA diractory:
        index = item_E.find("_") 
        sub_number = item_E[:index] #find its subject number by index of the first "_"
        try:
            val = float(tar[tar["EDA_num"] == int(sub_number)]["binary[%]"].iloc[0])
        except: # For some reason it gave me exceptions for some of the dataframes of val.
            continue
        else:
            if val > 25:# If the binary % is bigger then 25% continue to the next item
                continue
            for item_n in os.listdir(path_noise): #then, search for the matching noise file by the subject number
                if item_n.startswith(sub_number+"_") and item_n.endswith("Binary.csv"):
                    noise= pd.read_csv(path_noise+"\\"+item_n) #read the right noise
                    EDA = pd.read_csv(path_EDA+"\\"+item_E) #read the right EDA file

                    noise["StartTime"] =  pd.to_datetime(noise["StartTime"], dayfirst=True) #convert the start time of the noise to datetime (because of the format it reading days and month in the oposite way so we need dayfirst=true)
                    noise["EndTime"] =  pd.to_datetime(noise["EndTime"], dayfirst=True) #convert the end time of the noise to datetime
                    EDA["GMT date"] = pd.to_datetime(EDA["GMT date"]) #convert the EDA (GMT) time to datetime
                    total_row_length=len(EDA.index) #count the total number of rows

                    noisy_parts = noise[noise["BinaryLabels"] ==-1][["StartTime", "EndTime"]] #search for the noisy parts
                    artifacts_length = int(len(noisy_parts.index)) #count how much "-1" ("bad signal") the file has
                    set_times = set() #make an empty set
                    for row in EDA.loc[2:,"GMT date"]: #for every row in the EDA file:
                        for index,i in noisy_parts.iterrows(): #check if the row in the EDA file is between the start and the end time of the noise
                            if i["StartTime"]<=row<=i["EndTime"]:
                                set_times.add(row) #if it does then add the 'bad' rows to the set

                    bool_list = list() #make an emty list which will contain the true(good) and false(bad) rows of the EDA file
                    for row in EDA.iterrows(): #for each row in the EDA file 
                        boolean = (row[1][3] not in set_times) #decise whether it is not in the 'bad' rows (1,3 because itterrows gives tuple of index and row with all of the columns)
                        bool_list.append(boolean) #add the trues and false to the empty list
                    clean_EDA = EDA.loc[np.array(bool_list)].drop(["Unnamed: 0","Unnamed: 0.1"], axis=1) #make a clean EDA file by subsetting the trues and false


                    target_path = os.path.join(clean_tar, item_E) #make a target path for the new clean EDA file
                    clean_EDA.to_csv(target_path)            #write the file

In [41]:
cleaning_EDA_bin(path_EDA, path_noise, clean_tar) #call function

In [43]:
def cleaning_EDA_mul(path_EDA, path_noise, clean_tar): #define cleaning function
    
    tar = pd.read_csv(target) # Read the statistics file
    for item_E in os.listdir(path_EDA): #for every item in the EDA diractory:
        index = item_E.find("_") 
        sub_number = item_E[:index] #find its subject number by index of the first "_"
        try:
            val = float(tar[tar["EDA_num"] == int(sub_number)]["multiple[%]"].iloc[0])
        except: # For some reason it gave me exceptions for some of the dataframes of val.
            continue
        else:
            if val > 25:# If the binary % is bigger then 25% continue to the next item
                continue
            for item_n in os.listdir(path_noise): #then, search for the matching noise file by the subject number
                if item_n.startswith(sub_number+"_") and item_n.endswith("Multiclass.csv"):
                    noise= pd.read_csv(path_noise+"\\"+item_n) #read the right noise
                    EDA = pd.read_csv(path_EDA+"\\"+item_E) #read the right EDA file

                    noise["StartTime"] =  pd.to_datetime(noise["StartTime"], dayfirst=True) #convert the start time of the noise to datetime (because of the format it reading days and month in the oposite way so we need dayfirst=true)
                    noise["EndTime"] =  pd.to_datetime(noise["EndTime"], dayfirst=True) #convert the end time of the noise to datetime
                    EDA["GMT date"] = pd.to_datetime(EDA["GMT date"]) #convert the EDA (GMT) time to datetime
                    total_row_length=len(EDA.index) #count the total number of rows

                    noisy_parts = noise[noise["MulticlassLabels"] ==-1][["StartTime", "EndTime"]] #search for the noisy parts
                    artifacts_length = int(len(noisy_parts.index)) #count how much "-1" ("bad signal") the file has
                    set_times = set() #make an empty set
                    for row in EDA.loc[2:,"GMT date"]: #for every row in the EDA file:
                        for index,i in noisy_parts.iterrows(): #check if the row in the EDA file is between the start and the end time of the noise
                            if i["StartTime"]<=row<=i["EndTime"]:
                                set_times.add(row) #if it does then add the 'bad' rows to the set

                    bool_list = list() #make an emty list which will contain the true(good) and false(bad) rows of the EDA file
                    for row in EDA.iterrows(): #for each row in the EDA file 
                        boolean = (row[1][3] not in set_times) #decise whether it is not in the 'bad' rows
                        bool_list.append(boolean) #add the trues and false to the empty list
                    clean_EDA = EDA.loc[np.array(bool_list)].drop(["Unnamed: 0","Unnamed: 0.1"], axis=1) #make a clean EDA file by subsetting the trues and false


                    target_path = os.path.join(clean_tar, item_E) #make a target path for the new clean EDA file
                    clean_EDA.to_csv(target_path)            #write the file

In [52]:
cleaning_EDA_mul(path_EDA, path_noise, clean_tar) #call function